 # K-nearest neighbors Project Tutorial

In this guided project we will learn how to build a simple movie recommender system using k-nearest neighbors algorithm.

This project contains 2 datasets with different features for the same 5000 movies, so you should merge them.


In [ ]:
!pip install -r ../requirements.txt

## Import Data

In [4]:
import pandas as pd 
import numpy as np 

In [26]:
movies = pd.read_csv('../data/raw/tmdb_5000_movies.csv')
credits = pd.read_csv('../data/raw/tmdb_5000_credits.csv')

In [12]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 19995 to 25975
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_companies  4803 non-null   object 
 9   production_countries  4803 non-null   object 
 10  release_date          4802 non-null   object 
 11  revenue               4803 non-null   int64  
 12  runtime               4801 non-null   float64
 13  spoken_languages      4803 non-null   object 
 14  status                4803 non-null   object 
 15  tagline         

In [13]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 19995 to 25975
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   4803 non-null   object
 1   cast    4803 non-null   object
 2   crew    4803 non-null   object
dtypes: object(3)
memory usage: 150.1+ KB


In [14]:
movies.head(10)

,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,
19995,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
285,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
206647,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
49026,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
49529,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
559,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,890871626,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576
38757,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,591794936,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330
99861,280000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/1

In [15]:
credits.head(10)

,title,cast,crew
movie_id,,,
19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
559,Spider-Man 3,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
38757,Tangled,"[{""cast_id"": 34, ""character"": ""Flynn Rider (vo...","[{""credit_id"": ""52fe46db9251416c91062101"", ""de..."
99861,Avengers: Age of Ultron,"[{""cast_id"": 76, ""character"": ""Tony Stark / Ir...","[{""credit_id"": ""55d5f7d4c3a3683e7e0016eb"", ""de..."
767,Harry Potter and the Half-Blood Prince,"[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...","[{""credit_id"": ""52fe4273c3a36847f801fab1"", ""de..."


In [41]:
#merge both dataset
df_movies = movies.merge(credits, on='title') 

In [42]:
#select only some columns
df_movies = df_movies[['movie_id','title','overview','genres','keywords','cast','crew']]


In [30]:
df_movies.sample(10)

,movie_id,title,overview,genres,keywords,cast,crew
2184,5915,Into the Wild,"The true story of top student and athlete, Chr...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...","[{""id"": 255, ""name"": ""male nudity""}, {""id"": 97...","[{""cast_id"": 15, ""character"": ""Christopher McC...","[{""credit_id"": ""57190512c3a3687b8c004a60"", ""de..."
3702,5689,The Blue Lagoon,Two small children and a ship's cook survive a...,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 12, ...","[{""id"": 566, ""name"": ""sexual identity""}, {""id""...","[{""cast_id"": 7, ""character"": ""Emmeline"", ""cred...","[{""credit_id"": ""52fe4418c3a36847f8081faf"", ""de..."
2626,133694,Promised Land,A salesman for a natural gas company experienc...,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1415, ""name"": ""small town""}, {""id"": 30...","[{""cast_id"": 5, ""character"": ""Steve Butler"", ""...","[{""credit_id"": ""52fe4bb9c3a368484e19697d"", ""de..."
1358,9776,Head of State,When a presidential candidate dies unexpectedl...,"[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 279, ""name"": ""usa""}, {""id"": 577, ""name...","[{""cast_id"": 1, ""character"": ""Mays Gilliam"", ""...","[{""credit_id"": ""52fe452bc3a36847f80c025b"", ""de..."
1192,32823,Get Him to the Greek,Pinnacle records has the perfect plan to get t...,"[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 179430, ""name"": ""aftercreditsstinger""}]","[{""cast_id"": 1, ""character"": ""Aaron Green"", ""c...","[{""credit_id"": ""553e2e76c3a3687db1002f94"", ""de..."
1173,1213,The Talented Mr. Ripley,Tom Ripley is a calculating young man who beli...,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...","[{""id"": 110, ""name"": ""venice""}, {""id"": 131, ""n...","[{""cast_id"": 24, ""character"": ""Tom Ripley"", ""c...","[{""credit_id"": ""57c75cc39251410a5600036f"", ""de..."
245,714,Tomorrow Never Dies,A deranged media mogul is staging internationa...,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...","[{""id"": 212, ""name"": ""london england""}, {""id"":...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""52fe426fc3a36847f801e40d"", ""de..."
1624,109421,Side Effects,A woman turns to prescription medication as a ...,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...","[{""id"": 4373, ""name"": ""fraud""}, {""id"": 9665, ""...","[{""cast_id"": 14, ""character"": ""Jonathan Banks""...","[{""credit_id"": ""52fe4ab4c3a36847f81dd435"", ""de..."
722,14836,Coraline,"When Coraline moves to an old house, she feels...","[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""id"": 1566, ""name"": ""dream""}, {""id"": 4484, ""...","[{""cast_id"": 1, ""character"": ""Coraline Jones (...","[{""credit_id"": ""552b6ca592514101f6005760"", ""de..."
3333,2011,Persepolis,"In 1970s Iran, Marjane 'Marji' Statrapi watche...","[{""id"": 16, ""name"": ""Animation""}, {""id"": 18, ""...","[{""id"": 393, ""name"": ""civil war""}, {""id"": 970,...","[{""cast_id"": 7, ""character"": ""Marjane adulte e...","[{""credit_id"": ""52fe432cc3a36847f803fc85"", ""de..."


## Clean Data

In [32]:
#check if duplicate 
df_movies.duplicated().sum()

0

In [34]:
#check if null value
df_movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [43]:
df_movies.dropna(inplace = True)

In [36]:
#see the json
df_movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [44]:
#transform the json to list
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [38]:
convert(df_movies.iloc[0].genres)

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [45]:
df_movies['genres'] = df_movies['genres'].apply(convert)
df_movies['keywords'] = df_movies['keywords'].apply(convert)
df_movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [47]:
#transform the json to list limit to 3 value
def convert3(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            L.append(i['name'])
        count +=1  
    return L

In [48]:
df_movies['cast'] = df_movies['cast'].apply(convert3)
df_movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [49]:
#get the director of the moview from crew
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [50]:
df_movies['crew'] = df_movies['crew'].apply(fetch_director)

In [51]:
df_movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [56]:
#collapse the value of the feature, remove space
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
        
    return " ".join(L1)

In [57]:
df_movies['cast'] = df_movies['cast'].apply(collapse)
df_movies['crew'] = df_movies['crew'].apply(collapse)
df_movies['genres'] = df_movies['genres'].apply(collapse)
df_movies['keywords'] = df_movies['keywords'].apply(collapse)

In [58]:
df_movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,SamWorthington ZoeSaldana SigourneyWeaver,JamesCameron


In [59]:
#create new feature join the clean feature
df_movies['tags'] = df_movies['overview']+df_movies['genres']+df_movies['keywords']+df_movies['cast']+df_movies['crew']

In [62]:
new_df = df_movies[['movie_id','title','tags']]

## Process

### Text vectorization

In [60]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000 ,stop_words='english')

In [63]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [64]:
vectors.shape

(4806, 5000)

In [ ]:
#5000 most frequently used words
cv.get_feature_names()

### Let's find the cosine_similarity among the movies. Go ahead and run the following code lines in your project to see the results.

In [66]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors).shape

(4806, 4806)

In [67]:
similarity = cosine_similarity(vectors)

In [68]:
similarity[0]

array([1.        , 0.06963106, 0.06700252, ..., 0.02594996, 0.02787473,
       0.        ])

In [69]:
sorted(list(enumerate(similarity[0])),reverse =True , key = lambda x:x[1])[1:6]

[(1216, 0.25144550304711916),
 (151, 0.24140227479263382),
 (3730, 0.24140227479263382),
 (1204, 0.2321035412742638),
 (582, 0.22852652225356157)]

### Finally, create a recommendation function based on the cosine_similarity. This function should recommend the 5 most similar movies.

In [70]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0] ##fetching the movie index
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate( distances)),reverse =True , key = lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [74]:
recommend('Avatar')

Aliens vs Predator: Requiem
Beowulf
Falcon Rising
Predators
Battle: Los Angeles
